In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, ParameterGrid
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
from sklearn.metrics import r2_score

Using TensorFlow backend.


In [2]:
X_train = pd.read_parquet("X/X_train.parquet")
X_test = pd.read_parquet("X/X_test.parquet")
Y_train = pd.read_parquet("Y/Y_train.parquet")
Y_test = pd.read_parquet("Y/Y_test.parquet")
X_train = X_train[['Built', 'Below Secundary', 'Income Median',
       'Population', 'Over 65', 'Over 65 Percentage', 'Over 85',
       'Over 85 Percentage', 'Over 50', 'Over 50 Percentage', 'Under 20',
       'Under 20 Percentage', 'Income Per Capita', 'Tourism', 'Area',
       'Built Area']]
X_test = X_test[['Built', 'Below Secundary', 'Income Median',
       'Population', 'Over 65', 'Over 65 Percentage', 'Over 85',
       'Over 85 Percentage', 'Over 50', 'Over 50 Percentage', 'Under 20',
       'Under 20 Percentage', 'Income Per Capita', 'Tourism', 'Area',
       'Built Area']]
Complete = pd.read_parquet("dataset.parquet")

In [3]:
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train))
X_test = pd.DataFrame(scalerX.transform(X_test))
scalerY = MinMaxScaler().fit(Y_train)
Y_train = pd.DataFrame(scalerY.transform(Y_train))
Y_test = pd.DataFrame(scalerY.transform(Y_test))

In [4]:
vars1 = []
for n in range(len(X_train.columns)):
    vars1.append(array(X_train[n]))

In [5]:
vars2 = []
for n in range(len(vars1)):
    aux = []
    for i in range(len(vars1[n])):
        if(i%24 == 0 or i%24 == 1):
            continue
        aux.append(array([vars1[n][i-2], vars1[n][i-1], vars1[n][i]]))
    vars2.append(aux)

In [6]:
x_t = []
for n in range(len(vars2[0])):
    x_t.append([a[n] for a in vars2])

In [7]:
valvars1 = []
for n in range(len(X_test.columns)):
    valvars1.append(array(X_test[n]))

In [8]:
valvars2 = []
for n in range(len(valvars1)):
    aux = []
    for i in range(len(valvars1[n])):
        if(i%4 == 0):
            aux.append(array([vars1[n][int(24*int(i/4))+22],vars1[n][int(24*int(i/4))+23], valvars1[n][i]]))
        elif(i%4 == 1):
            aux.append(array([vars1[n][int(24*int(i/4))+23],valvars1[n][i-1], valvars1[n][i]]))
        else:
            aux.append(array([valvars1[n][i-2], valvars1[n][i-1], valvars1[n][i]]))
    valvars2.append(aux)

In [9]:
len(valvars2)

16

In [10]:
len(vars2)

16

In [11]:
x_val = []
for n in range(len(valvars2[0])):
    x_val.append([a[n] for a in valvars2])

In [12]:
y_t = [array(Y_train.loc[i]) for i in Y_train.index if not(i%24 == 0 or i%24 == 1)]

In [13]:
y_val = [array(Y_test.loc[i]) for i in Y_test.index]

In [52]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(7, activation='relu'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(array(x_t), array(y_t), epochs=400, verbose=2)

Epoch 1/400
 - 3s - loss: 0.0615
Epoch 2/400
 - 1s - loss: 0.0441
Epoch 3/400
 - 1s - loss: 0.0428
Epoch 4/400
 - 1s - loss: 0.0389
Epoch 5/400
 - 1s - loss: 0.0303
Epoch 6/400
 - 1s - loss: 0.0294
Epoch 7/400
 - 1s - loss: 0.0289
Epoch 8/400
 - 1s - loss: 0.0255
Epoch 9/400
 - 1s - loss: 0.0242
Epoch 10/400
 - 1s - loss: 0.0234
Epoch 11/400
 - 1s - loss: 0.0226
Epoch 12/400
 - 1s - loss: 0.0226
Epoch 13/400
 - 1s - loss: 0.0218
Epoch 14/400
 - 1s - loss: 0.0220
Epoch 15/400
 - 1s - loss: 0.0218
Epoch 16/400
 - 1s - loss: 0.0222
Epoch 17/400
 - 1s - loss: 0.0244
Epoch 18/400
 - 1s - loss: 0.0210
Epoch 19/400
 - 1s - loss: 0.0209
Epoch 20/400
 - 1s - loss: 0.0194
Epoch 21/400
 - 1s - loss: 0.0188
Epoch 22/400
 - 1s - loss: 0.0203
Epoch 23/400
 - 1s - loss: 0.0182
Epoch 24/400
 - 1s - loss: 0.0219
Epoch 25/400
 - 1s - loss: 0.0190
Epoch 26/400
 - 1s - loss: 0.0176
Epoch 27/400
 - 1s - loss: 0.0175
Epoch 28/400
 - 1s - loss: 0.0179
Epoch 29/400
 - 1s - loss: 0.0160
Epoch 30/400
 - 1s - lo

 - 1s - loss: 0.0033
Epoch 239/400
 - 1s - loss: 0.0032
Epoch 240/400
 - 1s - loss: 0.0034
Epoch 241/400
 - 1s - loss: 0.0045
Epoch 242/400
 - 1s - loss: 0.0039
Epoch 243/400
 - 1s - loss: 0.0032
Epoch 244/400
 - 1s - loss: 0.0032
Epoch 245/400
 - 1s - loss: 0.0036
Epoch 246/400
 - 1s - loss: 0.0032
Epoch 247/400
 - 1s - loss: 0.0033
Epoch 248/400
 - 1s - loss: 0.0035
Epoch 249/400
 - 1s - loss: 0.0032
Epoch 250/400
 - 1s - loss: 0.0034
Epoch 251/400
 - 1s - loss: 0.0034
Epoch 252/400
 - 1s - loss: 0.0036
Epoch 253/400
 - 1s - loss: 0.0037
Epoch 254/400
 - 1s - loss: 0.0032
Epoch 255/400
 - 1s - loss: 0.0034
Epoch 256/400
 - 1s - loss: 0.0031
Epoch 257/400
 - 1s - loss: 0.0033
Epoch 258/400
 - 1s - loss: 0.0030
Epoch 259/400
 - 1s - loss: 0.0030
Epoch 260/400
 - 1s - loss: 0.0032
Epoch 261/400
 - 1s - loss: 0.0030
Epoch 262/400
 - 1s - loss: 0.0031
Epoch 263/400
 - 1s - loss: 0.0031
Epoch 264/400
 - 1s - loss: 0.0031
Epoch 265/400
 - 1s - loss: 0.0035
Epoch 266/400
 - 1s - loss: 0.0033

In [48]:
y_pred = model.predict(array(x_val))

In [49]:
r2 = r2_score(y_val, y_pred)
print(r2)

-5.336760375903991


In [50]:
y_pred

array([[0.04735708, 0.06846496, 0.06977582, ..., 0.05069843, 0.03190689,
        0.9011245 ],
       [0.04903656, 0.07096703, 0.07355767, ..., 0.05073814, 0.03422658,
        0.9053912 ],
       [0.04996429, 0.07037004, 0.07501271, ..., 0.05119531, 0.04043676,
        0.89829844],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.9924503 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.9411409 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.87834466]], dtype=float32)

In [51]:
y_val

[array([0.05461928, 0.06296563, 0.06282874, 0.08993096, 0.03834536,
        0.0414233 , 1.04347826]),
 array([0.05560656, 0.08518318, 0.06062632, 0.09113083, 0.03834536,
        0.04476404, 1.08695652]),
 array([0.05560656, 0.08518318, 0.06062632, 0.09113083, 0.03834536,
        0.04476404, 1.13043478]),
 array([0.05560656, 0.08518318, 0.06062632, 0.09113083, 0.03834536,
        0.04476404, 1.17391304]),
 array([0.01983136, 0.0291223 , 0.03959393, 0.02001394, 0.07029826,
        0.00133958, 1.04347826]),
 array([1.98460665e-02, 2.90493598e-02, 4.05229782e-02, 2.01683935e-02,
        7.11791718e-02, 9.91055654e-04, 1.08695652e+00]),
 array([2.02344139e-02, 2.98287976e-02, 4.14847382e-02, 2.03846286e-02,
        7.46085852e-02, 9.12378534e-04, 1.13043478e+00]),
 array([2.06093121e-02, 3.02077902e-02, 4.27947825e-02, 2.10772312e-02,
        7.54235420e-02, 7.10164532e-04, 1.17391304e+00]),
 array([1.95350523e-02, 2.71707717e-02, 4.12814249e-02, 2.58506615e-02,
        4.56941076e-02, 5.86